In [1]:
from nltk.lm.preprocessing import padded_everygram_pipeline

sentences = [
    ["я", "вижу", "стол"],
    ["я", "вижу", "стул"],
    ["я", "вижу", "сыр"],
    ["я", "ем", "сыр"],
    ["я", "стол"]
]

n = 2
n_grams, words = padded_everygram_pipeline(n, sentences)

print([list(sent) for sent in n_grams])
# Список всех слов подряд, с уже добавленными словами <s> </s>
print(list(words))

# внимание, второй проход по n_grams, words невозможен
print(list(words))  # пусто!

[[('<s>',), ('<s>', 'я'), ('я',), ('я', 'вижу'), ('вижу',), ('вижу', 'стол'), ('стол',), ('стол', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'я'), ('я',), ('я', 'вижу'), ('вижу',), ('вижу', 'стул'), ('стул',), ('стул', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'я'), ('я',), ('я', 'вижу'), ('вижу',), ('вижу', 'сыр'), ('сыр',), ('сыр', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'я'), ('я',), ('я', 'ем'), ('ем',), ('ем', 'сыр'), ('сыр',), ('сыр', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'я'), ('я',), ('я', 'стол'), ('стол',), ('стол', '</s>'), ('</s>',)]]
['<s>', 'я', 'вижу', 'стол', '</s>', '<s>', 'я', 'вижу', 'стул', '</s>', '<s>', 'я', 'вижу', 'сыр', '</s>', '<s>', 'я', 'ем', 'сыр', '</s>', '<s>', 'я', 'стол', '</s>']
[]


In [2]:
from nltk.lm import MLE

n = 2
n_grams, words = padded_everygram_pipeline(n, sentences)
model = MLE(n) # Модель MLE означает отсутствие сглаживания
model.fit(n_grams, words)  # обучаем

Теперь внутри `model` содержится таблица вероятностей $P(u|w)$.


In [3]:
model.score('вижу', ['я'])  # вероятность "вижу" после слов 'я'

0.6

!!! Это как наши $\frac35$!!
А что для 3-грамм?

In [4]:
n = 3
n_grams, words = padded_everygram_pipeline(n, sentences)
model = MLE(n) # Модель MLE означает отсутствие сглаживания
model.fit(n_grams, words)  # обучаем

print(model.score('сыр', ['я', 'вижу']))
print(model.score('вижу', ['я']))

0.3333333333333333
0.6


## Сглаживание Lidstone
при $\lambda=1$ это называют сглаживанием Лапласа

In [16]:
from nltk.lm import Lidstone

n = 2
n_grams, words = padded_everygram_pipeline(n, sentences)
model = Lidstone(1, n)  # добавление 1 в числителе, n размер n-грамм
model.fit(n_grams, words)  # обучаем

print(model.score('вижу', ['я']))
print(4/14) # кроме наших 13 слов есть еще волшебное слово UNKN (неизвестное)


0.2857142857142857
0.2857142857142857


Для оценки модели вычислим P(другого текста)


In [6]:
test_sentence = ["я", "вижу"]

from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams

n = 2
padded_sentence = pad_both_ends(test_sentence, n)  # добавляем <s>
print(list(padded_sentence))

padded_sentence = pad_both_ends(test_sentence, n)  # добавляем <s>
padded_sentence_ngrams = ngrams(padded_sentence, n)
print(list(padded_sentence_ngrams))

padded_sentence = pad_both_ends(test_sentence, n)  # добавляем <s>
padded_sentence_ngrams = ngrams(padded_sentence, n)
print(model.perplexity(padded_sentence_ngrams))

['<s>', 'я', 'вижу', '</s>']
[('<s>', 'я'), ('я', 'вижу'), ('вижу', '</s>')]
4.610436292058448


## Сглаживание Kneyser-Ney


In [22]:
from nltk.lm import KneserNeyInterpolated
model = KneserNeyInterpolated(n, 0.1)  # вычитаем 0.1 из числителя
# далее аналогично Lidstone
n = 2
n_grams, words = padded_everygram_pipeline(n, sentences)
model.fit(n_grams, words)
print(model.score('вижу', ['я']))

0.5866666666666667


## Вычисление качества модели, perplexity


In [12]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams

n = 2
# сначала нужно добавить символы начала и конца предложения
sentence = ['мы', 'слова', 'одного', 'предложения']
padded_sentence = pad_both_ends(sentence, n)
print(list(padded_sentence))

# после распечатки padded_sentence стух, поэтому создадим его заново
padded_sentence = pad_both_ends(sentence, n)
all_sentence_n_grams = ngrams(padded_sentence, n)
print(list(all_sentence_n_grams))

['<s>', 'мы', 'слова', 'одного', 'предложения', '</s>']
[('<s>', 'мы'), ('мы', 'слова'), ('слова', 'одного'), ('одного', 'предложения'), ('предложения', '</s>')]


Необходимо для каждого предложения того корпуса, на котором мы оцениваем качество, собрать n-граммы этим способом. Общую массу всех n-грамм передать в `model.perplexity()`. Perplexity должен быть как можно меньше.


Какие гиперпараметры: n = 1, 2, 3, 4, 5 (решаем отдельно для каждого n, потому что для разных n получаются разные модели с несравнимыми perplexity).
                      Lidstone: 0.01 0.05 0.1 0.2 0.5 1 2 5
                      KneyserNey: 0.01 0.02 0.05 0.1 0.2 0.5 0.95 0.99

In [23]:
padded_sentence = pad_both_ends(sentence, n)
all_sentence_n_grams = ngrams(padded_sentence, n)
model.perplexity(all_sentence_n_grams)

19.680517330979004

Весь код для вычисления perplexity


In [30]:
sentences = [
    ["я", "вижу", "стол"],
    ["я", "вижу", "стул"],
    ["я", "вижу", "сыр"],
    ["я", "ем", "сыр"],
    ["я", "стол"]
]

n_grams, words = padded_everygram_pipeline(n, sentences)
model = KneserNeyInterpolated(n, 0.1)
model.fit(n_grams, words)

sentence = ['мы', 'слова', 'одного', 'предложения']
padded_sentence = pad_both_ends(sentence, n)
all_sentence_n_grams = ngrams(padded_sentence, n)
model.perplexity(all_sentence_n_grams)

sentence1 = ['мы', 'слова', 'одного', 'предложения']
sentence2 = ['еще', 'слова', 'другого', 'предложения']
ngrams1 = ngrams(pad_both_ends(sentence1, n), n)
ngrams2 = ngrams(pad_both_ends(sentence2, n), n)

all_ngrams = list(ngrams1) + list(ngrams2)
print(all_ngrams)
model.perplexity(all_ngrams)

[('<s>', 'мы'), ('мы', 'слова'), ('слова', 'одного'), ('одного', 'предложения'), ('предложения', '</s>'), ('<s>', 'еще'), ('еще', 'слова'), ('слова', 'другого'), ('другого', 'предложения'), ('предложения', '</s>')]


19.68051733097903